This document provides the EWA score calculations

In [22]:
# Load data
import pandas as pd
import numpy as np
from datetime import datetime
import time
pd.options.mode.chained_assignment = None  # default='warn'
player_DF = pd.read_pickle('data_all_TSE')
player_info = pd.read_pickle('info_all_TSE') 

In [23]:
print(player_DF.head())
print(player_info.head())

  playerid round gameid    payoff globalid                strats  \
0       26     1      1  0.100000       15  [0.0, 0.0, 0.0, 1.0]   
1       26     2      1  7.813333       15  [0.0, 0.5, 0.5, 0.0]   
2       26     3      1  7.250000       15  [0.0, 0.0, 1.0, 0.0]   
3       26     4      1  8.231944       15  [0.0, 0.5, 0.5, 0.0]   
4       26     5      1  9.366667       15  [0.0, 0.0, 1.0, 0.0]   

                   self_avg_s  \
0  [0.146, 0.306, 0.248, 0.3]   
1     [0.13, 0.61, 0.26, 0.0]   
2   [0.075, 0.35, 0.575, 0.0]   
3    [0.18, 0.38, 0.42, 0.02]   
4     [0.23, 0.3, 0.42, 0.05]   

                                      opponent_avg_s  \
0  [0.15, 0.38722222222222225, 0.4461111111111111...   
1  [0.16666666666666666, 0.39666666666666667, 0.3...   
2  [0.225, 0.3333333333333333, 0.325, 0.116666666...   
3  [0.3416666666666667, 0.35555555555555557, 0.30...   
4  [0.13125, 0.33958333333333335, 0.5291666666666...   

                    prev_self  \
0                     

In [24]:
# Calculates payoff given mixed strategy by the vector s, round t

def calc_payoff(s_idx, t, player_info, player_DF):
    nr = player_info.n_strats
    self_s = np.zeros(nr)
    self_s[s_idx] = 1
    opp_s = np.array(player_DF['opponent_avg_s'][player_DF['round']==t].values[0])
    mat = player_info['payoff_mat']
    return np.dot(np.dot(self_s,mat),opp_s.T)

In [27]:
# The attention function

def delta(t, s_idx, player_info, player_DF):
    val = player_DF.payoff[player_DF['round']==t].values
    δ=0
    if calc_payoff(s_idx,t, player_info, player_DF)>=float(val):
        δ=1
    return δ

In [28]:
# The change-dectecting decay rate

def phi_rate(t, player_info, player_DF):
    S_t=0
    for m in range(0,len(player_DF.opponent_avg_s.iloc[0])):
        h_t = [sum(i) for i in zip(*player_DF.opponent_avg_s[player_DF['round']<=t])][m]/len(player_DF['round'][player_DF['round']<=t])
        r_t=player_DF.opponent_avg_s[player_DF['round']==t].values[0][m]
        S_t += (h_t-r_t)**2
    return 1-S_t/2

In [36]:
# Calculate attractions and experience weights
def calc_attraction(s_idx, t, player_info, player_DF, para, N_prev, A_prev = None, endog = False):
    κ = para[0]
    ϕ = para[1]
    δ = para[2]
    λ = para[3]
    t_1 = player_DF['round'].loc[player_DF['round']<=t-1].iloc[-1]
    if endog==1:
        N = phi_rate( t_1, player_info, player_DF) * ( 1 - κ ) * N_prev + 1
        d_t = delta( t_1, s_idx, player_info, player_DF)
        A_1 = phi_rate( t_1, player_info, player_DF) * N_prev * A_prev
        A_2 = ( d_t + ( 1 - d_t )*player_DF['strats'].loc[player_DF['round']==t].iloc[0][s_idx] ) * calc_payoff(s_idx, t, player_info, player_DF) 
        A = ( A_1 + A_2 ) / N
    else:
        N = ϕ *( 1 - κ ) * N_prev + 1
        A = ϕ * N_prev * A_prev + ( δ + (1-δ)*player_DF['strats'].loc[player_DF['round']==t].iloc[0][s_idx]) * calc_payoff(s_idx, t, player_info, player_DF)
        A = A / N
    return [A, N]

In [70]:
# Add attractions A[t] to dataframe

def add_Ats_to_DF(player_info, player_DF, para, endog = False):
    nr = player_info.n_strats.iloc[0]
    A = [np.ones(nr) / nr]
    N = [1]
    for t in player_DF['round'].iloc[1:]:
        As = np.ones(nr)
        for s in range(0, nr):   
            As[s] = calc_attraction( s, t, player_info.iloc[0], player_DF, para, N[-1], A[-1][s], endog)[0]
        A.append(As)
        N.append(calc_attraction( 0, t, player_info.iloc[0], player_DF, para, N[-1], A[-1][0], endog)[1])
    return N, A

In [66]:
#Logits

def add_logits_to_DF(player_info, player_DF, λ):
    Out = []
    nr = player_info.n_strats.iloc[0]
    for t in range(0,len(player_DF['round'].unique())):
        As = player_DF['A'].iloc[t]
        tot_log = 0
        vec = np.zeros(nr)
        for s in range(0, nr):
            if np.exp(λ*As[s]) > 10**30:
                vec[s] = 10**30
                vec[s].astype(np.longdouble)
            else:
                vec[s] = np.exp(λ*As[s]).astype(np.longdouble)
            tot_log += vec[s]
        Out.append(vec/tot_log)
    return Out

In [75]:
#get average distance for all players all games
def calc_tot_distance(sessions, games, para, endog = False):
    dist = 0
    for sid in sessions:
        for gid in games:
            for pid in player_DF.playerid[(player_DF['gameid']==gid) & (player_DF['session']==sid)].unique():
                DF_t = player_DF[(player_DF['gameid']==gid) & (player_DF['playerid']==pid) & (player_DF['session']==sid)]
                info_t = player_info[(player_info['gameid']==gid) & (player_info['playerid']==pid) & (player_info['session']==sid)]
                DF_t['N'] = add_Ats_to_DF( info_t, DF_t, para, endog)[0]
                DF_t['A'] = add_Ats_to_DF( info_t, DF_t, para, endog)[1]
                DF_t['log_probs'] = add_logits_to_DF(info_t, DF_t, para[3])
                dist += np.sum(np.sum((DF_t['strats']-DF_t['log_probs'])**2))
    return dist

In [80]:
#Simple test
import warnings
warnings.filterwarnings("ignore")
para = [0,0,1,1]
calc_tot_distance([0],[1], [1,1,0,40], 0)

77.84201966765228

In [19]:
# Generate a min function for different parameter combinations
# self-tuning EWA (endog=1)

def max_func(sessions, games, λ):
    para = [0,0,1,λ]  
    return calc_tot_distance(sessions, games, para, 1)

In [38]:
# δ=1, κ=0, λ and ϕ free weighted stochastic fictious play 
def max_func(sessions, games, par):
    λ = par[0]
    ϕ = par[1]
    para = [0,ϕ,1,λ]  
    return calc_tot_distance(sessions, games, para, 0)

In [ ]:
# λ free, δ=1, ϕ =κ=0 qantal response cournot 
def max_func(sessions, games, λ):
    para = [0,0,1,λ]  
    return calc_tot_distance(sessions, games, para, 0)

In [77]:
# λ and ϕ free, δ=0 and κ=1 is comulative reinforcement model
def max_func(sessions, games, par):
    λ = par[0]
    ϕ = par[1]
    para = [1,ϕ,0,λ]  
    return calc_tot_distance(sessions, games, para, 0)

In [ ]:
# All free EWA full power
def max_func(sessions, games, par):
    return calc_tot_distance(sessions, games, para, 0)

In [81]:
# Finding optimal parameters for 7-fold cross validation all games together
from scipy.optimize import minimize
import functools
bnds = ((0.001,50),(0,1))
for t in np.arange(7):
    a = np.arange(7)
    b = np.arange(6)+1
    a = np.delete(a, t)
    current_time = time.time()
    dt = datetime.fromtimestamp(current_time).strftime("%A, %B %d, %Y %I:%M:%S")
    print(f"Session 7-{t} at {dt}")
    score = minimize(functools.partial(max_func,a,b), x0=(0.1,0.1), method='SLSQP', bounds=bnds)
    print(f"It took {int((time.time() - current_time)/60)} minutes")
    score = pd.DataFrame(score)
    #score.to_pickle(f"data/opt_wstficp_{t}") done
    #score.to_pickle(f"data/opt_lambda_qr7_{t}")
    #score.to_pickle(f"data/opt_lambda_stewa_{t}") done
    #score.to_pickle(f"data/opt_crfm_{t}") done
    score.to_pickle(f"data/opt_ewa_{t}")

Session 7-0 at Saturday, December 12, 2020 12:01:53
It took 64 minutes
Session 7-1 at Saturday, December 12, 2020 01:05:55
It took 55 minutes
Session 7-2 at Saturday, December 12, 2020 02:01:23
It took 59 minutes
Session 7-3 at Saturday, December 12, 2020 03:01:04
It took 57 minutes
Session 7-4 at Saturday, December 12, 2020 03:58:22
It took 60 minutes
Session 7-5 at Saturday, December 12, 2020 04:59:02
It took 61 minutes
Session 7-6 at Saturday, December 12, 2020 06:00:15
It took 57 minutes


In [ ]:
# Finding optimal parameters for 7-fold cross validation by games

from scipy.optimize import minimize
import functools
bnds = ((0.001,50),(0,1))
for t in np.arange(7):
    a = np.arange(7)
    a = np.delete(a, t)
    for gid in player_DF.gameid.unique():
        current_time = time.time()
        dt = datetime.fromtimestamp(current_time).strftime("%A, %B %d, %Y %I:%M:%S")
        print(f"Session 7-{t} at {dt} game {gid}")
        score = minimize(functools.partial(max_func,a, [gid]), x0=[0.1,0.1], method='SLSQP', bounds=bnds)
        print(f"It took {int((time.time() - current_time)/60)} minutes")
        score = pd.DataFrame(score)
        #score.to_pickle(f"data/opt_wstficp_game{gid}_ses7_{t}") done
        #score.to_pickle(f"data/opt_lambda_qr_game{gid}_ses7_{t}")
        #score.to_pickle(f"data/opt_lambda_stewa_game{gid}_ses7_{t}") done
        #score.to_pickle(f"data/opt_crfm_game{gid}_ses_{t}") done
        score.to_pickle(f"data/opt_ewa_game{gid}_ses_{t}")

Session 7-0 at Saturday, December 12, 2020 06:57:24 game 1
It took 11 minutes
Session 7-0 at Saturday, December 12, 2020 07:09:06 game 2
It took 9 minutes
Session 7-0 at Saturday, December 12, 2020 07:18:31 game 3
It took 8 minutes
Session 7-0 at Saturday, December 12, 2020 07:27:10 game 4
It took 5 minutes
Session 7-0 at Saturday, December 12, 2020 07:32:11 game 5
It took 11 minutes
Session 7-0 at Saturday, December 12, 2020 07:43:14 game 6
It took 13 minutes
Session 7-1 at Saturday, December 12, 2020 07:57:02 game 1
It took 10 minutes
Session 7-1 at Saturday, December 12, 2020 08:07:58 game 2
It took 9 minutes
Session 7-1 at Saturday, December 12, 2020 08:17:41 game 3
It took 8 minutes
Session 7-1 at Saturday, December 12, 2020 08:26:03 game 4
It took 5 minutes
Session 7-1 at Saturday, December 12, 2020 08:31:20 game 5
It took 10 minutes
Session 7-1 at Saturday, December 12, 2020 08:41:26 game 6
It took 13 minutes
Session 7-2 at Saturday, December 12, 2020 08:55:09 game 1
It took 11 

In [36]:
# Benchmarks
import random
random.seed(30)
d